PURPOSE:
------------------------------
This creates a list of icustayIDs of patients who develop sepsis at some point 
in the ICU. records charttime for onset of sepsis. Uses sepsis3 criteria

STEPS:
-------------------------------
IMPORT DATA FROM CSV FILES  
FLAG PRESUMED INFECTION  
PREPROCESSING  
REFORMAT in 4h time slots  
COMPUTE SOFA at each time step  
FLAG SEPSIS  

note: the process generates the same features as the final MDP dataset, most of which are not used to compute SOFA

# Import all data

In [594]:
import pandas as pd
abx = pd.read_csv('/Users/faaiz/exportdir/abx.csv', sep='|')
culture = pd.read_csv('/Users/faaiz/exportdir/culture.csv', sep='|')
microbio = pd.read_csv('/Users/faaiz/exportdir/microbio.csv', sep='|')
demog = pd.read_csv('/Users/faaiz/exportdir/demog.csv', sep='|')
ce010 = pd.read_csv('/Users/faaiz/exportdir/ce010000.csv', sep='|')
ce1020 = pd.read_csv('/Users/faaiz/exportdir/ce1000020000.csv', sep='|')
ce2030 = pd.read_csv('/Users/faaiz/exportdir/ce2000030000.csv', sep='|')
ce3040 = pd.read_csv('/Users/faaiz/exportdir/ce3000040000.csv', sep='|')
ce4050 = pd.read_csv('/Users/faaiz/exportdir/ce4000050000.csv', sep='|')
ce5060 = pd.read_csv('/Users/faaiz/exportdir/ce5000060000.csv', sep='|')
ce6070 = pd.read_csv('/Users/faaiz/exportdir/ce6000070000.csv', sep='|')
ce7080 = pd.read_csv('/Users/faaiz/exportdir/ce7000080000.csv', sep='|')
ce8090 = pd.read_csv('/Users/faaiz/exportdir/ce8000090000.csv', sep='|')
ce90100 = pd.read_csv('/Users/faaiz/exportdir/ce90000100000.csv', sep='|')
lab_ce = pd.read_csv('/Users/faaiz/exportdir/labs_ce.csv', sep='|').rename(columns = {'charttime': 'timestp'}, inplace = False)
lab_le = pd.read_csv('/Users/faaiz/exportdir/labs_le.csv', sep='|')
labU = pd.concat([lab_ce, lab_le], ignore_index = True)
MV = pd.read_csv('/Users/faaiz/exportdir/mechvent.csv', sep='|')
inputpreadm = pd.read_csv('/Users/faaiz/exportdir/preadm_fluid.csv', sep='|')
inputMV = pd.read_csv('/Users/faaiz/exportdir/fluid_mv.csv', sep='|')
inputCV = pd.read_csv('/Users/faaiz/exportdir/fluid_cv.csv', sep='|')
vasoMV = pd.read_csv('/Users/faaiz/exportdir/vaso_mv.csv', sep='|')
vasoCV = pd.read_csv('/Users/faaiz/exportdir/vaso_cv.csv', sep='|')
UOpreadm = pd.read_csv('/Users/faaiz/exportdir/preadm_uo.csv', sep='|')
UO = pd.read_csv('/Users/faaiz/exportdir/uo.csv', sep='|')

reflabs = pd.read_csv('/Users/faaiz/exportdir/Reflabs.csv', header=None)
refvitals = pd.read_csv('/Users/faaiz/exportdir/Refvitals.csv', header=None)
sample_and_hold = pd.read_csv('/Users/faaiz/exportdir/sample_and_hold.csv')

# Initial Data Manipulations

In [595]:
# Change the headers of sample_and_hold to remove ''' '''
for i in sample_and_hold:
    sample_and_hold.rename(columns={i:i.replace("'","")}, inplace=True)
    
sample_and_hold.head()

,Height_cm,Weight_kg,GCS,RASS,HR,SysBP,MeanBP,DiaBP,RR,SpO2,...,Arterial_pH,paO2,paCO2,Arterial_BE,Arterial_lactate,HCO3,ETCO2,SvO2,mechvent,extubated
0,168,72,6,6,2,2,2,2,2,2,...,8,8,8,8,8,8,8,8,6,6


In [596]:
# if charttime is empty but chartdate isn't
microbio.loc[microbio['charttime'].isnull(), 'charttime'] = microbio['chartdate'] 
microbio = microbio.drop(columns = 'chartdate')
bacterio = pd.concat([microbio, culture], ignore_index = True)

In [597]:
demog['morta_90'] = demog['morta_90'].fillna(0)
demog['morta_hosp'] = demog['morta_hosp'].fillna(0)
demog['elixhauser'] = demog['elixhauser'].fillna(0)

In [598]:
inputMV.head()
inputMV['normrate'] = inputMV['rate']*inputMV['tev']/inputMV['amount']

In [599]:
def find_icustay_id_from_demog(hadm_id, time, subject_id = None):
    if subject_id is not None:
        df = demog.loc[demog['subject_id'] == subject_id]
    else:
        df = demog.loc[demog['hadm_id'] == hadm_id]
    for index, row in df.iterrows():
        if (time >= row['intime'] - 48*3600 and time <= row['outtime'] + 48*3600) or len(df) == 1:
            return row['icustay_id']
    df2 = df.loc[df['hadm_id'] == hadm_id]
    if len(df2) == 1:
        return df2['icustay_id'].values[0]
    return None

In [600]:
# TODO: to be removed!
len(bacterio)
len(bacterio.loc[bacterio['icustay_id'].isnull()])

2003

In [601]:
bacterio.head()
# Fill in missing ICUSTAY IDs in bacterio
for index, row in bacterio.iterrows():
    if pd.isna(row['icustay_id']):
        charttime, hadm_id, subject_id = row['charttime'], row['hadm_id'], row['subject_id']
        icustayid = find_icustay_id_from_demog(hadm_id, charttime, subject_id)
        if icustayid is not None:
            bacterio.at[index,'icustay_id'] = icustayid

In [602]:
# Fill in missing ICUSTAY IDs in bacterio
for index, row in abx.iterrows():
    if pd.isna(row['icustay_id']):
        time, hadm_id = row['startdate'], row['hadm_id']
        icustayid = find_icustay_id_from_demog(hadm_id, time)
        if icustayid is not None:
            abx.at[index,'icustay_id'] = icustayid

# Find presumed onset of infection according to sepsis3 guidelines

In [603]:
from sklearn.metrics.pairwise import euclidean_distances

onset = pd.DataFrame(columns=['subject_id', 'icustay_id', 'onsettime'])

for icustayid in range(1,100001):
    ab = pd.Series.to_numpy(abx.loc[abx['icustay_id'] == icustayid + 200000, 'startdate'])
    bact = pd.Series.to_numpy(bacterio.loc[bacterio['icustay_id'] == icustayid + 200000, 'charttime'])
    subj_bact = pd.Series.to_numpy(bacterio.loc[bacterio['icustay_id'] == icustayid + 200000, 'subject_id'])
    
    if len(ab) > 0 and len(bact) > 0:
        D = euclidean_distances(ab.reshape(-1,1), bact.reshape(-1,1))/3600
        for i in range(len(D)):
            M = min(D[i])
            I = D[i].argmin()
            ab1 = ab[i]
            bact1 = bact[I]
            
            if M <= 24 and ab1 <= bact1:
                onset = onset.append({'subject_id': subj_bact[0], 'icustay_id': icustayid, 'onsettime': ab1}, ignore_index=True)
                break
            elif M <= 72 and ab1 >= bact1:
                onset = onset.append({'subject_id': subj_bact[0], 'icustay_id': icustayid, 'onsettime': bact1}, ignore_index=True)
                break
        

In [604]:
len(onset.loc[onset['onsettime']>0])

65

# Replacing item_ids with column numbers from reference tables

In [605]:
def find_the_col_number(df, num):
    for index, row in df.iterrows():
        arr = pd.Series.to_numpy(row)
        if num in arr:
            return int(index+1)
        
def replace_itemids_with_col_num(df, reftable):
    for index, row in df.iterrows():
        df.at[index, 'colnum'] = find_the_col_number(reftable, row['itemid'])
    df.drop(columns=['itemid'], inplace=True)
    df['colnum'] = df['colnum'].astype({'colnum': 'int64'})

In [606]:
replace_itemids_with_col_num(labU, reflabs)
replace_itemids_with_col_num(ce010,refvitals)
replace_itemids_with_col_num(ce1020,refvitals)
replace_itemids_with_col_num(ce2030,refvitals)
replace_itemids_with_col_num(ce3040,refvitals)
replace_itemids_with_col_num(ce4050,refvitals)
replace_itemids_with_col_num(ce5060,refvitals)
replace_itemids_with_col_num(ce6070,refvitals)
replace_itemids_with_col_num(ce7080,refvitals)
replace_itemids_with_col_num(ce8090,refvitals)
replace_itemids_with_col_num(ce90100,refvitals)

# Initial reformat with chartevents, labs and mechvent

gives an array with all unique charttime (1 per row) and all items in columns.
## IMPORTANT 
Here i use -48 -> +24 because that's for sepsis3 cohort defintion!!  
I need different time period for the MDP (-24 -> +48)

In [607]:
winb4=49   #lower limit for inclusion of data (48h before time flag)
winaft=25  # upper limit (24h after)
irow = 0
reformat, qstime = pd.DataFrame(), pd.DataFrame()

for icustayid in range(1,100001):
    qst = onset.loc[onset['icustay_id'] == icustayid, 'onsettime']
    if len(qst) > 0 and qst.values[0] > 0:
        d1 = demog.loc[demog['icustay_id'] == icustayid + 200000, ['age', 'dischtime']]
        
        if len(d1['age']) > 0 and d1['age'].values[0] > 6574:
            if icustayid < 10000:
                temp = ce010.loc[ce010['icustay_id'] == icustayid + 200000]
            elif icustayid < 20000:
                temp = ce1020.loc[ce1020['icustay_id'] == icustayid + 200000]
            elif icustayid < 30000:
                temp = ce2030.loc[ce2030['icustay_id'] == icustayid + 200000]
            elif icustayid < 40000:
                temp = ce3040.loc[ce3040['icustay_id'] == icustayid + 200000]
            elif icustayid < 50000:
                temp = ce4050.loc[ce4050['icustay_id'] == icustayid + 200000]
            elif icustayid < 60000:
                temp = ce5060.loc[ce5060['icustay_id'] == icustayid + 200000]
            elif icustayid < 70000:
                temp = ce6070.loc[ce6070['icustay_id'] == icustayid + 200000]
            elif icustayid < 80000:
                temp = ce7080.loc[ce7080['icustay_id'] == icustayid + 200000]
            elif icustayid < 90000:
                temp = ce8090.loc[ce8090['icustay_id'] == icustayid + 200000]
            else:
                temp = ce90100.loc[ce90100['icustay_id'] == icustayid + 200000]
            
            temp = temp.loc[temp['charttime'].between(qst.values[0]-(winb4+4)*3600,qst.values[0]+(winaft+4)*3600)] #time period of interest
            
            # LABEVENTS
            temp2 = labU.loc[labU['icustay_id'] == icustayid + 200000]
            temp2 = temp2.loc[temp2['timestp'].between(qst.values[0]-(winb4+4)*3600,qst.values[0]+(winaft+4)*3600)] #time period of interest
            
            #MECH VENT and Exubated
            temp3 = MV.loc[MV['icustay_id'] == icustayid + 200000]
            temp3 = temp3.loc[temp3['charttime'].between(qst.values[0]-(winb4+4)*3600, qst.values[0]+(winaft+4)*3600)] #time period of interest
            
            unique_timestp = pd.concat([temp['charttime'], temp2['timestp'].rename('charttime'), temp3['charttime']]).unique()
            unique_timestp.sort() # list of unique timestamps from all 3 sources sorted in ascending order
            
            for i in range(len(unique_timestp)):
                timedata = temp.loc[temp['charttime'] == unique_timestp[i]]
                
                reformat.at[irow, 'timestep'] = i
                reformat.at[irow, 'icustay_id'] = icustayid
                reformat.at[irow, 'charttime'] = unique_timestp[i]
                
                if len(timedata) > 0:
                    col = timedata['colnum'].values[0]
                    value = timedata['valuenum'].values[0]
                    reformat.at[irow, 3 + col] = value

                # LAB values
                timedata = temp2.loc[temp2['timestp'] == unique_timestp[i]]
                if len(timedata) > 0:
                    col = timedata['colnum'].values[0]
                    value = timedata['valuenum'].values[0]
                    reformat.at[irow, 31+col] = value;

                # MV
                value = temp3.loc[temp3['charttime'] == unique_timestp[i], ['mechvent', 'extubated']]
                if len(value) > 0:
                    reformat.at[irow, 'mechvent'] = value['mechvent'].values[0]
                    reformat.at[irow, 'extubated'] = value['extubated'].values[0]
                
                irow = irow + 1
            
            if len(unique_timestp) > 0:
                qstime = qstime.append({'icustay_id': icustayid, 'firsttimestp': unique_timestp[0], 'lasttimestp':unique_timestp[-1], 'dischtime': d1['dischtime'].values[0]}, ignore_index = True)

reformat['timestep'] = reformat['timestep'].astype({'timestep': 'int64'})
reformat['icustay_id'] = reformat['icustay_id'].astype({'icustay_id': 'int64'})
            

Map the column numbers in reformat to the corresponding column headers

In [608]:
sah_headers = np.array(sample_and_hold.columns)

for i in range(4,69):
    if i in reformat:
        reformat.rename(columns={i: sah_headers[i-4]}, inplace=True)

# Outliers

In [609]:
def deloutbelow(df, var, thres):
    if var in df:
        for index, row in df.iterrows():
            if row[var] < thres:
                df.at[index, var] = None
            
def deloutabove(df, var, thres):
    if var in df:
        for index, row in df.iterrows():
            if row[var] > thres:
                df.at[index, var] = None

In [610]:
# Weight
deloutabove(reformat,"Weight_kg",300)
# HR
deloutabove(reformat,"HR",250)
# BP
deloutabove(reformat,"SysBP",300)
deloutbelow(reformat,"MeanBP",0)
deloutabove(reformat,"MeanBP",200)
deloutbelow(reformat,"DiaBP",0)
deloutabove(reformat,"DiaBP",200)
# RR
deloutabove(reformat,"RR",80)
# Sp02
deloutabove(reformat,"SpO2",150);
if "SpO2" in reformat:
    reformat.loc[reformat["SpO2"] > 100, "SpO2"] = 100
# temp
if "Temp_C" in reformat and "Temp_F" in reformat:
    reformat.loc[(reformat["Temp_C"] > 90) & (reformat["Temp_F"].isnull()), "Temp_F"] = reformat["Temp_C"]
deloutabove(reformat,"Temp_C",90)
# Fi02
deloutabove(reformat,"FiO2_100",100)
if "FiO2_100" in reformat:
    reformat.loc[reformat["FiO2_100"] < 1, "FiO2_100"] = reformat["FiO2_100"]*100
deloutbelow(reformat,"FiO2_100",20)
deloutabove(reformat,"FiO2_1",1.5)
# O2 Flow
deloutabove(reformat,"O2flow",70)
# PEEP
deloutbelow(reformat,"PEEP",0)
deloutabove(reformat,"PEEP",40)
# TV
deloutabove(reformat,"TidalVolume",1800)
# MV
deloutabove(reformat,"MinuteVentil",50)
# K+
deloutbelow(reformat,"Potassium",1)
deloutabove(reformat,"Potassium",15)
# Na
deloutbelow(reformat,"Sodium",95)
deloutabove(reformat,"Sodium",178)
# Cl
deloutbelow(reformat,"Chloride",70)
deloutabove(reformat,"Chloride",150)
# Glc
deloutbelow(reformat,"Glucose",1)
deloutabove(reformat,"Glucose",1000)
# Creat
deloutabove(reformat,"Creatinine",150)
# Mg
deloutabove(reformat,"Magnesium",10)
# Ca
deloutabove(reformat,"Calcium",20)
# ionized Ca
deloutabove(reformat,"Ionised_Ca",5)
# CO2
deloutabove(reformat,"CO2_mEqL",120)
# SGPT/SGOT
deloutabove(reformat,"SGOT",10000)
deloutabove(reformat,"SGPT",10000)
# Hb/Ht
deloutabove(reformat,"Hb",20)
deloutabove(reformat,"Ht",65)
# WBC
deloutabove(reformat,"WBC_count",500)
# plt
deloutabove(reformat,"Platelets_count",2000)
# INR
deloutabove(reformat,"INR",20)
# pH
deloutbelow(reformat,"Arterial_pH",6.7)
deloutabove(reformat,"Arterial_pH",8)
# po2
deloutabove(reformat,"paO2",700)
# pco2
deloutabove(reformat,"paCO2",200)
# BE
deloutbelow(reformat,"Arterial_BE",-50)
# lactate
deloutabove(reformat,"Arterial_lactate",30)

Some more manipulation/imputation from existing values

In [611]:
# estimate GCS from RASS - data from Wesley JAMA 2003
def SAH(temp, vitalslab_hold):
    oldstayid = temp['icustay_id'].values[0]
    lastcharttime = {}
    lastvalue = {}
    
    for col in temp:
        if col in vitalslab_hold:

            for index, row in temp.iterrows():
                if oldstayid != row['icustay_id']:
                    lastcharttime = {}
                    lastvalue = {}
                    oldstayid = row['icustay_id']

                if not pd.isna(temp.at[index, col]):
                    lastcharttime[col] = temp.at[index, 'charttime']
                    lastvalue[col] = temp.at[index, col]

                if index > 0:
                    if pd.isna(temp.at[index, col]) and temp.at[index, 'icustay_id'] == oldstayid and (temp.at[index, 'charttime'] - lastcharttime.get(col,0)) <= vitalslab_hold.at[0,col]*3600:
                        temp.at[index, col] = lastvalue.get(col, 0)
            

In [612]:
# Sample and Hold on raw data
SAH(reformat, sample_and_hold) 

# Data combination

In [613]:
# WARNING: the time window of interest has been defined above (here -48 -> +24)

timestep = 4 # resolution of timesteps, in hours
icustayidlist = pd.unique(reformat['icustay_id'])
num_of_patients = len(icustayidlist)
reformat2 = pd.DataFrame()

for i in range(num_of_patients):
    icustayid = icustayidlist[i] # 1 to 100,000 not 200 to 300k
    
    temp = reformat.loc[reformat['icustay_id'] == icustayid]
    first_timestp = temp.loc[temp['timestep'] == 0, 'charttime'].values[0]
        
    # IV Fluid stuff
    inputMV_subset = inputMV.loc[inputMV['icustay_id'] == icustayid + 200000]
    inputCV_subset = inputCV.loc[inputCV['icustay_id'] == icustayid + 200000]
    
    starttime = inputMV_subset['starttime']
    endtime = inputMV_subset['endtime']
    rate = inputMV_subset['normrate']
    
    preadmission = inputpreadm.loc[inputpreadm['icustay_id'] == icustayid + 200000, 'inputpreadm']
    if len(preadmission)>0:
        total_volume = preadmission.sum()
    else:
        total_volume = 0
        
    # compute volume of fluid given before start of record!
    t0 = 0
    t1 = first_timestp
    
    # input from MV (4 ways to compute)
    infusion = (rate*(endtime-starttime)*((endtime <= t1) & (starttime >= t0))/3600 +\
                rate*(endtime - t0)*((starttime <= t0) & (endtime <= t1) & (endtime >= t0))/3600 +\
                rate*(t1 - starttime)*((starttime >= t0) & (endtime >= t1) & (starttime <= t1))/3600 +\
                rate*(t1-t0)*((endtime >= t1) & (starttime <= t0))/3600).sum()
    # all boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
    bolus = (inputMV_subset.loc[pd.isna(inputMV_subset['rate']) & (inputMV_subset['starttime'] >= t0) & (inputMV_subset['starttime'] <= t1), 'tev'].sum() +\
             inputCV_subset.loc[(inputCV_subset['charttime'] >= t0) & (inputCV_subset['charttime'] <= t1), 'tev'].sum())
    
    total_volume = total_volume + infusion + bolus
    
    # Vasopressors
    vasoMV_subset = vasoMV.loc[vasoMV['icustay_id'] == icustayid + 200000]
    vasoCV_subset = vasoCV.loc[vasoCV['icustay_id'] == icustayid + 200000]
    start_vp = vasoMV_subset['starttime'] #start of vp infusion
    end_vp = vasoMV_subset['endtime']
    rate_vp = vasoMV_subset['rate_std']
    
    # DEMOGRAPHICS / gender, age, elixhauser, re-admit, died in hosp?, died within
    # 48h of out_time (likely in ICU or soon after), died within 90d after admission?
    demog_subset = demog.loc[demog['icustay_id'] == icustayid + 200000]
    qstime_subset = qstime.loc[qstime['icustay_id'] == icustayid]
    if len(demog_subset) > 0:
        demographics = {'gender':demog_subset['gender'].values[0],\
                        'age': demog_subset['age'].values[0],\
                        'elixhauser':demog_subset['elixhauser'].values[0],\
                        're_admission': int(demog_subset['adm_order'].values[0] > 1),\
                        'died_in_hosp': demog_subset['morta_hosp'].values[0],\
                        'died_within_48h_of_out_time': int(abs(demog_subset['dod'].values[0] - demog_subset['outtime'].values[0]) < 24*3600*2),\
                        'mortality_90d': demog_subset['morta_90'].values[0]}
    if len(qstime_subset) > 0:
        demographics.update({'delay_end_of_record_and_discharge_or_death': (qstime_subset['lasttimestp'].values[0] - qstime_subset['firsttimestp'].values[0])/3600})
    
    # URINE OUTPUT
    UO_subset = UO.loc[UO['icustay_id'] == icustayid + 200000]
    preadmission = UOpreadm.loc[UOpreadm['icustay_id'] == icustayid, 'value'] #preadmission UO
    if len(preadmission) > 0:
        UO_total = preadmission.sum()
    else:
        UO_total = 0
        
    UOnow = UO_subset.loc[(UO_subset['charttime'] >= t0) & (UO_subset['charttime'] >= t1), 'value'].sum()
    UO_total = UO_total + UOnow
    
    for j in range(0,80,timestep):
        t0 = 3600*j+ first_timestp
        t1 = 3600*(j + timestep)+ first_timestp
        temp_subset = temp.loc[temp['charttime'].between(t0,t1)]
        if len(temp_subset) > 0:
            data = {'bloc': (j/timestep)+1,\
                    'icustay_id': icustayid,\
                    'charttime': t0}
            data.update(demographics)
            data.update(temp_subset.drop(columns = ['timestep', 'icustay_id', 'charttime']).mean().to_dict())            
            
            
            #VASOPRESSORS
            # for CV: dose at timestamps.
            # for MV: 4 possibles cases, each one needing a different way to compute the dose of VP actually administered:
            #----t0---start----end-----t1----
            #----start---t0----end----t1----
            #-----t0---start---t1---end
            #----start---t0----t1---end----
            
            # MV
            ratemv = vasoMV_subset.loc[(vasoMV_subset['endtime'].between(t0,t1)) |\
                                      ((vasoMV_subset['starttime'] >= t0) & (vasoMV_subset['endtime'] <= t1)) |\
                                      (vasoMV_subset['starttime'].between(t0,t1)) |\
                                      ((vasoMV_subset['starttime'] <= t0) & (vasoMV_subset['endtime'] >= t1)), 'rate_std']
            #CV
            ratecv = vasoCV_subset.loc[vasoCV_subset['charttime'].between(t0,t1), 'rate_std']
            median_rate = pd.concat([ratemv,ratecv]).median()
            max_rate = pd.concat([ratemv,ratecv]).max()
            data.update({'median_dose_vaso': median_rate, 'max_dose_vaso': max_rate})
            
            # MV
            # input from MV (4 ways to compute)
            infusion = (rate*(endtime-starttime)*((endtime <= t1) & (starttime >= t0))/3600 +\
                        rate*(endtime - t0)*((starttime <= t0) & (endtime <= t1) & (endtime >= t0))/3600 +\
                        rate*(t1 - starttime)*((starttime >= t0) & (endtime >= t1) & (starttime <= t1))/3600 +\
                        rate*(t1-t0)*((endtime >= t1) & (starttime <= t0))/3600).sum()
            # all boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
            bolus = (inputMV_subset.loc[pd.isna(inputMV_subset['rate']) & (inputMV_subset['starttime'] >= t0) & (inputMV_subset['starttime'] <= t1), 'tev'].sum() +\
                     inputCV_subset.loc[(inputCV_subset['charttime'] >= t0) & (inputCV_subset['charttime'] <= t1), 'tev'].sum())

            total_volume = total_volume + infusion + bolus
            
            data.update({'input_total': total_volume, 'input_4hourly': infusion + bolus})
            
            # UO
            UOnow = UO_subset.loc[(UO_subset['charttime'] >= t0) & (UO_subset['charttime'] >= t1), 'value'].sum()
            UO_total = UO_total + UOnow
            
            data.update({'output_total': UO_total, 'output_4hourly': UOnow, 'cumulated_balance': total_volume - UO_total})
            reformat2 = reformat2.append(data, ignore_index=True)
                                                                                                          

If values have less than 70% missing values (over 30% of values present): We keep them

In [623]:
for col in reformat2:
    miss = reformat2[col].isnull().sum() / len(reformat2)
    if miss > 0.7:
        reformat2.drop(columns=col, inplace = True)

reformat2.sort_values(by=['icustay_id', 'bloc'], inplace=True)

# Handling of missing values

In [639]:
reformat3 = reformat2
attribute_cols = ['bloc', 'icustay_id', 'charttime', 'gender', 'age', 'elixhauser', 're_admission', 'died_in_hosp', 'died_within_48h_of_out_time', 'mortality_90d', 'delay_end_of_record_and_discharge_or_death', 'extubated', 'mechvent']
for col in reformat2:
    if col not in attribute_cols:
        miss = reformat2[col].isnull().sum()/len(reformat2)
        if miss > 0 and miss <= 0.05:
            reformat3[col] = reformat3[col].interpolate(method='linear').fillna(0)
            

In [675]:
# KNN Imputation - Done on chunks of 10K records
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)

value_cols = set(reformat3.columns) - set(attribute_cols)
ref_values = reformat3[value_cols].copy()
ind = 0
while(ind < len(ref_values)):
    ref_values.loc[ind:ind+10000, value_cols] = imputer.fit_transform(ref_values.loc[ind:ind+10000, value_cols])
    ind = ind + 10000

reformat4 = pd.concat(axis = 1, objs = [reformat3[attribute_cols], ref_values])   


Compute some derived variables: P/F, Shock Index, SOFA, SIRS...

In [683]:
# Correct gender
reformat4['gender'] = reformat4['gender']-1

In [684]:
reformat4.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,Weight_kg,input_4hourly,HCO3,RR,input_total,SysBP,output_total,Arterial_BE,Hb,SGPT
0,1.0,1006.0,4.330306e+09,0.0,25154.170995,3.0,0.0,1.0,1.0,1.0,...,74.200000,0.000000,23.0,13.000000,0.000000,178.000000,8166.0,17.000000,10.2,12.0
1,3.0,1006.0,4.330335e+09,0.0,25154.170995,3.0,0.0,1.0,1.0,1.0,...,74.200000,0.000000,21.0,11.000000,0.000000,137.000000,12249.0,0.000000,8.5,36.0
2,6.0,1006.0,4.330378e+09,0.0,25154.170995,3.0,0.0,1.0,1.0,1.0,...,73.900002,275.000000,21.0,27.666667,275.000000,134.000000,15932.0,0.000000,12.6,36.0
3,1.0,1204.0,4.794583e+09,1.0,29424.868472,5.0,0.0,0.0,0.0,1.0,...,80.500000,36.666667,27.0,14.857143,5036.666667,156.769231,5025.0,-0.791667,8.9,125.8
4,2.0,1204.0,4.794597e+09,1.0,29424.868472,5.0,0.0,0.0,0.0,1.0,...,80.500000,672.054217,23.0,11.000000,5708.720883,93.000000,7140.0,-1.947368,8.9,16.0
